In [1]:
import pandas as pd
import numpy as np
from nltk.probability import FreqDist
import nltk
from collections import Counter

from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data=pd.read_excel('Data_Science_Internship_Assignment_1.xlsx',sheet_name='Data')

In [3]:
data.head()

,NAME,WEBSITE,TAGLINE,HQ REGION,HQ COUNTRY,HQ CITY,TAGS,LAUNCH_YEAR,LAUNCH_MONTH,GROWTH STAGE,LINKEDIN,TYPE
0,63336,http://63336.com,Ai-enabled q&a service that answers to various...,Europe,United Kingdom,London,mobile,2002,September,late growth stage,NaN,NaN
1,@Futsal,http://futsaluk.net,Educational courses through the medium of spor...,Europe,United Kingdom,Birmingham,NaN,2008,NaN,early growth stage,https://www.linkedin.com/company/-futsal-group...,NaN
2,#5 Magazine,http://5mag.co,Multi-platform digital lifestyle magazines abo...,Europe,United Kingdom,London,publishing;branding;media;platform;entertainment,2007,NaN,NaN,NaN,NaN
3,03Numbers,http://planet-numbers.co.uk/numbers/0333,Planet Numbers are the leading provider of 03 ...,Europe,United Kingdom,NaN,NaN,2008,September,early growth stage,http://www.linkedin.com/company/planet-numbers,NaN
4,077football News & Media,http://077football.com,"077Football - the deep-rooted, hyperlocal foot...",Europe,United Kingdom,Cambridge,sport;advertising;football;network;game develo...,2009,NaN,late growth stage,https://www.linkedin.com/company/077football-n...,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11582 entries, 0 to 11581
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   NAME          11582 non-null  object 
 1   WEBSITE       11582 non-null  object 
 2   TAGLINE       11453 non-null  object 
 3   HQ REGION     11582 non-null  object 
 4   HQ COUNTRY    11582 non-null  object 
 5   HQ CITY       10908 non-null  object 
 6   TAGS          9593 non-null   object 
 7   LAUNCH_YEAR   11582 non-null  int64  
 8   LAUNCH_MONTH  2695 non-null   object 
 9   GROWTH STAGE  8584 non-null   object 
 10  LINKEDIN      8596 non-null   object 
 11  TYPE          0 non-null      float64
dtypes: float64(1), int64(1), object(10)
memory usage: 1.1+ MB


In [5]:
data1=data.drop(['HQ CITY','GROWTH STAGE','LINKEDIN','LAUNCH_MONTH'],axis=1)

In [6]:
#Rename Columns for easy access and no inconsistency in future
data1.rename(columns={"HQ REGION":'HQ_REGION','LAUNCH DATE':"LAUNCH_DATE"},inplace=True)

In [7]:
data1.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE
0,63336,http://63336.com,Ai-enabled q&a service that answers to various...,Europe,United Kingdom,mobile,2002,NaN
1,@Futsal,http://futsaluk.net,Educational courses through the medium of spor...,Europe,United Kingdom,NaN,2008,NaN
2,#5 Magazine,http://5mag.co,Multi-platform digital lifestyle magazines abo...,Europe,United Kingdom,publishing;branding;media;platform;entertainment,2007,NaN
3,03Numbers,http://planet-numbers.co.uk/numbers/0333,Planet Numbers are the leading provider of 03 ...,Europe,United Kingdom,NaN,2008,NaN
4,077football News & Media,http://077football.com,"077Football - the deep-rooted, hyperlocal foot...",Europe,United Kingdom,sport;advertising;football;network;game develo...,2009,NaN


In [8]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11582 entries, 0 to 11581
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   NAME         11582 non-null  object 
 1   WEBSITE      11582 non-null  object 
 2   TAGLINE      11453 non-null  object 
 3   HQ_REGION    11582 non-null  object 
 4   HQ COUNTRY   11582 non-null  object 
 5   TAGS         9593 non-null   object 
 6   LAUNCH_YEAR  11582 non-null  int64  
 7   TYPE         0 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 724.0+ KB


**ISSUE** : Tagline and Tags are two important attributes to find entity type of a company. However there are four type of data available here:
                1. Both Tagline and Tags are available
                2. Only Tagline available
                3. Only Tags available
                4. Companies which started before 1990
                
So, let's divide datasets into 3 and use only data sets where either one or both are available. It means that we have to drop 45 companies from our this dataset for our analysis. And we will be identifying data of only 11537 companies.

In [9]:
# Datasets with only Tags
data2=data1[(data.LAUNCH_YEAR>=1990)& (data1.TAGS.notna())&(data1.TAGLINE.isna())]

In [10]:
# Datasets with only Tagline
data3=data1[(data1.TAGLINE.notna()) & (data1.TAGS.isna())&(data1.LAUNCH_YEAR>=1990)]

In [11]:
#Dataset with both Tags and Tagline
data4=data1[(data1.TAGLINE.notna()) & (data.TAGS.notna())&(data1.LAUNCH_YEAR>=1990)]

In [12]:
data4.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE
0,63336,http://63336.com,Ai-enabled q&a service that answers to various...,Europe,United Kingdom,mobile,2002,NaN
2,#5 Magazine,http://5mag.co,Multi-platform digital lifestyle magazines abo...,Europe,United Kingdom,publishing;branding;media;platform;entertainment,2007,NaN
4,077football News & Media,http://077football.com,"077Football - the deep-rooted, hyperlocal foot...",Europe,United Kingdom,sport;advertising;football;network;game develo...,2009,NaN
5,077football News & Media Limited,http://077football.is,"A football (soccer) centric, community-specifi...",Europe,United Kingdom,publishing;game development;social;advertising...,2009,NaN
6,1 Click Services Limited,http://1clickcloud.net,"We provide out of the box hosted solutions, ma...",Europe,United Kingdom,cloud technology,2008,NaN


In [13]:
data5=data1[(data1.LAUNCH_YEAR<1990) & ((data1.TAGS.notna()) | (data1.TAGLINE.notna())) ]

## Let's start with companies whose only Tags are available

In [14]:
data2.LAUNCH_YEAR=data2.LAUNCH_YEAR.astype(int)

C:\Users\giris\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [15]:
data2. info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 100 to 11385
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   NAME         84 non-null     object 
 1   WEBSITE      84 non-null     object 
 2   TAGLINE      0 non-null      object 
 3   HQ_REGION    84 non-null     object 
 4   HQ COUNTRY   84 non-null     object 
 5   TAGS         84 non-null     object 
 6   LAUNCH_YEAR  84 non-null     int32  
 7   TYPE         0 non-null      float64
dtypes: float64(1), int32(1), object(6)
memory usage: 5.6+ KB


In [16]:
data2.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE
100,4Sight Financial Software,http://4sight.com,NaN,Europe,United Kingdom,fintech,2003,NaN
135,A&D Sales Limited,http://adsalesltd.com,NaN,Europe,United Kingdom,storage;industrial technologies;fuel,1998,NaN
141,a1-computer-repairs,http://a1-computer-repairs.co.uk,NaN,Europe,United Kingdom,local,2000,NaN
283,Adaptive Lighting,https://www.aptechnologies.co.uk/suppliers/ada...,NaN,Europe,United Kingdom,industry;lighting,2008,NaN
713,App-Ologies,http://app-ologies.com,NaN,Europe,United Kingdom,mobile,2010,NaN


In [17]:
data2['TAGS2']=data2.TAGS.str.split(';')

C:\Users\giris\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
data2.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE,TAGS2
100,4Sight Financial Software,http://4sight.com,NaN,Europe,United Kingdom,fintech,2003,NaN,[fintech]
135,A&D Sales Limited,http://adsalesltd.com,NaN,Europe,United Kingdom,storage;industrial technologies;fuel,1998,NaN,"[storage, industrial technologies, fuel]"
141,a1-computer-repairs,http://a1-computer-repairs.co.uk,NaN,Europe,United Kingdom,local,2000,NaN,[local]
283,Adaptive Lighting,https://www.aptechnologies.co.uk/suppliers/ada...,NaN,Europe,United Kingdom,industry;lighting,2008,NaN,"[industry, lighting]"
713,App-Ologies,http://app-ologies.com,NaN,Europe,United Kingdom,mobile,2010,NaN,[mobile]


In [19]:
TAGS=[]
for i in data2.TAGS2:
    for j in i:
        TAGS.append(j)

In [20]:
c=Counter(TAGS)

In [21]:
c.most_common()

[('fintech', 19),
 ('mobile', 14),
 ('media', 12),
 ('software', 11),
 ('video', 9),
 ('design', 8),
 ('retail', 7),
 ('social', 7),
 ('search engine', 7),
 ('app', 6),
 ('navigation', 6),
 ('e-commerce', 6),
 ('seo', 6),
 ('data', 5),
 ('entertainment', 5),
 ('sound technology', 5),
 ('branding', 5),
 ('deep tech', 5),
 ('local', 4),
 ('game development', 4),
 ('consulting services', 4),
 ('platform', 4),
 ('regtech compliance', 4),
 ('arts & culture', 4),
 ('payment', 4),
 ('advertising', 4),
 ('social media', 4),
 ('adtech', 4),
 ('finance', 4),
 ('lighting', 3),
 ('testing', 3),
 ('compliance', 3),
 ('messaging', 3),
 ('ft 1000', 3),
 ('party', 3),
 ('shopping', 3),
 ('mapping', 3),
 ('cleantech', 3),
 ('construction', 3),
 ('wealth management', 2),
 ('geopositioning', 2),
 ('automated technology', 2),
 ('cloud technology', 2),
 ('commercial real estate', 2),
 ('collaboration', 2),
 ('office space', 2),
 ('team collaboration', 2),
 ('workspace', 2),
 ('management', 2),
 ('innovatio

In [22]:
education=['education','certification','21st century skills','human resources']
government=['sharing economy','personal health','carbon','electric energy','charity','fuel','industry','lighting','verification',
           'compliance','arts & culture','banking','monitoring','mapping','employment','treacking' ,'positioning','research and development',
           'heating','thermal','communication','navigation','geopositioning','social','media','risk','telecommunications','theatre','music',
           'tracking']
others=['hotel','local','localisation','real estate','wholesale','printing','design','space','office space','asset management',
       'financial services','finance','accommodation','shopping','vitamin','lbs','accounting','energy storage','wearable devices',
       'construction','back office','adventure','insurance','brokerage','trading','fittings','installation','investing','storage',
       'consulting services','wealth management','retail','testing','commercial real estate','management','publishing','branding',
       'party','delivery','thread','pr','chat','collaboration','workspace','risk management','advertising','ride','messaging',
       'hardware','ft 1000','team collaboration']


In [23]:
non_tech=education+government+others

In [24]:
all_items=list(c)

In [25]:
tech=[]
for x in all_items:
    if x not in non_tech:
        tech.append(x)

In [26]:
def industry(x,tech,government,education,other):
    if len(set(x)&set(tech))>0:
        value='Startup'
    elif len(set(x)&set(education))>0:
        value='Universities/Schools'
    elif len(set(x)&set(government))>0:
        value='Government/Non-Profit'
    else:
        value='Unclassified'
    return value
    

In [27]:
data2['TYPE']=data2.TAGS2.apply(industry,args=(tech,government,education,others))

C:\Users\giris\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
data2.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE,TAGS2
100,4Sight Financial Software,http://4sight.com,NaN,Europe,United Kingdom,fintech,2003,Startup,[fintech]
135,A&D Sales Limited,http://adsalesltd.com,NaN,Europe,United Kingdom,storage;industrial technologies;fuel,1998,Startup,"[storage, industrial technologies, fuel]"
141,a1-computer-repairs,http://a1-computer-repairs.co.uk,NaN,Europe,United Kingdom,local,2000,Unclassified,[local]
283,Adaptive Lighting,https://www.aptechnologies.co.uk/suppliers/ada...,NaN,Europe,United Kingdom,industry;lighting,2008,Government/Non-Profit,"[industry, lighting]"
713,App-Ologies,http://app-ologies.com,NaN,Europe,United Kingdom,mobile,2010,Startup,[mobile]


## Data with only Taglines

In [29]:
data3.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE
1,@Futsal,http://futsaluk.net,Educational courses through the medium of spor...,Europe,United Kingdom,NaN,2008,NaN
3,03Numbers,http://planet-numbers.co.uk/numbers/0333,Planet Numbers are the leading provider of 03 ...,Europe,United Kingdom,NaN,2008,NaN
12,1000heads,https://1000heads.com/,Help brands to be more social; to talk to peop...,Europe,United Kingdom,NaN,2000,NaN
15,10Associates,http://10associates.co.uk,Over 200 Successful Re-brands,Europe,United Kingdom,NaN,2004,NaN
17,111pix,http://111pix.com,Uk-based independent production and internatio...,Europe,United Kingdom,NaN,2006,NaN


In [30]:
data3['TAGS2']=data3.TAGLINE.apply(lambda x:word_tokenize(x))

C:\Users\giris\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
data3.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE,TAGS2
1,@Futsal,http://futsaluk.net,Educational courses through the medium of spor...,Europe,United Kingdom,NaN,2008,NaN,"[Educational, courses, through, the, medium, o..."
3,03Numbers,http://planet-numbers.co.uk/numbers/0333,Planet Numbers are the leading provider of 03 ...,Europe,United Kingdom,NaN,2008,NaN,"[Planet, Numbers, are, the, leading, provider,..."
12,1000heads,https://1000heads.com/,Help brands to be more social; to talk to peop...,Europe,United Kingdom,NaN,2000,NaN,"[Help, brands, to, be, more, social, ;, to, ta..."
15,10Associates,http://10associates.co.uk,Over 200 Successful Re-brands,Europe,United Kingdom,NaN,2004,NaN,"[Over, 200, Successful, Re-brands]"
17,111pix,http://111pix.com,Uk-based independent production and internatio...,Europe,United Kingdom,NaN,2006,NaN,"[Uk-based, independent, production, and, inter..."


In [32]:
def lower(x):
    value=[i.lower() for i in x ]
    return value

In [33]:
data3.TAGS2=data3.TAGS2.apply(lower)

C:\Users\giris\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [34]:
data3.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE,TAGS2
1,@Futsal,http://futsaluk.net,Educational courses through the medium of spor...,Europe,United Kingdom,NaN,2008,NaN,"[educational, courses, through, the, medium, o..."
3,03Numbers,http://planet-numbers.co.uk/numbers/0333,Planet Numbers are the leading provider of 03 ...,Europe,United Kingdom,NaN,2008,NaN,"[planet, numbers, are, the, leading, provider,..."
12,1000heads,https://1000heads.com/,Help brands to be more social; to talk to peop...,Europe,United Kingdom,NaN,2000,NaN,"[help, brands, to, be, more, social, ;, to, ta..."
15,10Associates,http://10associates.co.uk,Over 200 Successful Re-brands,Europe,United Kingdom,NaN,2004,NaN,"[over, 200, successful, re-brands]"
17,111pix,http://111pix.com,Uk-based independent production and internatio...,Europe,United Kingdom,NaN,2006,NaN,"[uk-based, independent, production, and, inter..."


In [35]:
def remove_stop_words(x):
    filtered=[]
    for w in x:
        if w not in stop_words:
                filtered.append(w)
    return filtered
        

In [36]:
data3['token_words']=data3.TAGS2.apply(remove_stop_words)

C:\Users\giris\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
data3.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE,TAGS2,token_words
1,@Futsal,http://futsaluk.net,Educational courses through the medium of spor...,Europe,United Kingdom,NaN,2008,NaN,"[educational, courses, through, the, medium, o...","[educational, courses, medium, sports, childre..."
3,03Numbers,http://planet-numbers.co.uk/numbers/0333,Planet Numbers are the leading provider of 03 ...,Europe,United Kingdom,NaN,2008,NaN,"[planet, numbers, are, the, leading, provider,...","[planet, numbers, leading, provider, 03, numbe..."
12,1000heads,https://1000heads.com/,Help brands to be more social; to talk to peop...,Europe,United Kingdom,NaN,2000,NaN,"[help, brands, to, be, more, social, ;, to, ta...","[help, brands, social, ;, talk, people, ,, tal..."
15,10Associates,http://10associates.co.uk,Over 200 Successful Re-brands,Europe,United Kingdom,NaN,2004,NaN,"[over, 200, successful, re-brands]","[200, successful, re-brands]"
17,111pix,http://111pix.com,Uk-based independent production and internatio...,Europe,United Kingdom,NaN,2006,NaN,"[uk-based, independent, production, and, inter...","[uk-based, independent, production, internatio..."


In [38]:
tags1=[]
for i in data3.token_words:
    for j in i:
        tags1.append(j)
print(tags1)

['educational', 'courses', 'medium', 'sports', 'children', 'aged', '16', '18', 'years', 'planet', 'numbers', 'leading', 'provider', '03', 'numbers', 'uk', 'require', 'non', 'geographical', '0330', '0333', 'number', 'business', ',', 'help', 'brands', 'social', ';', 'talk', 'people', ',', 'talked', 'people', '200', 'successful', 're-brands', 'uk-based', 'independent', 'production', 'international', 'sales', 'company', 'formed', 'filmmaker', ',', 'actor', 'online', 'estate', 'agent', 'helping', 'sell', 'property', 'low', 'fixed', 'fee', 'commission', 'rightmove', ',', 'zoopla', 'savings', 'photos', 'consumers', 'allowing', 'edit', 'purchase', 'photos', 'online', 'secured', 'website', 'london', 'online', 'fashion', 'training', 'academy', 'help', 'aspiring', 'fashion', 'stylists', 'get', 'professionally', 'trained', '360', 'thinking', 'promote', 'businesse', '360fusion', 'specialise', '360°', 'virtual', 'tours', ',', 'web', 'design', 'development', 'flash', 'applications', 'online', 'media'

In [39]:
fdist=FreqDist(tags1)
print(fdist.most_common())

[(',', 906), ('services', 222), ('company', 214), ('uk', 146), ('solutions', 144), ('business', 136), ('online', 127), ('&', 125), ('digital', 109), ('marketing', 97), ('provider', 94), ('development', 83), ('agency', 79), ('technology', 79), ('products', 77), ('design', 76), ('service', 76), ('leading', 74), ('management', 73), ('london', 72), ('based', 68), ('software', 65), ('web', 60), ('industry', 55), ('providing', 53), ('businesses', 51), ('global', 51), ('ltd', 51), ('specialist', 50), ('systems', 48), ('one', 48), ('support', 47), ('independent', 45), ('world', 45), ('market', 44), ('’', 44), ('provide', 43), ('people', 42), ('-', 42), ('home', 42), ('|', 41), ('website', 40), ('quality', 40), ("'s", 40), ('(', 40), (')', 40), ('range', 40), ('social', 38), ('network', 38), ('group', 37), ('high', 37), ('information', 37), ('international', 36), ('training', 36), ('offering', 36), ('consultancy', 36), ('operates', 36), ('mobile', 36), ('firm', 35), ('commercial', 35), ('media'

In [40]:
tech1=['online','digital','short-codes','technology','e2e','e-learning','ebaystoredesign','software','datasets','website','funddata.com','tools','social','network','innovative','data','internet','seo','technologies',
      'application','websites','innovation','technical','ecommerce','cloud','integration','e-commerce','transformation','analytics',
      'intelligence','startups','database','biotechnology','intelligent','revolutionary','saas','email','programmes','cyber',
      'fast-growing','hugedomains.com','web-based','browser','namecheap.com','sensors','application','visualisation','game','hardware',
      'twitter','digitally','databases','itunes','spotify','tablets','gadget','connectivity','analytical','integrator','technica',
      'audiolock','customer-centric','bio-processing','booster','visualization','morewithanalytics','companiesandmarketscom',
      'programming','analyse','technology-bensions','entrepreneurial','digitalair','www.fileswire.com','e-base','computing',
      'android-powered','pc','scans','videoconferencing','innovations','datacoms','funddata.com','programme','groupola.com',
      'clean-tech','system-on-a-chip','indiconews.com','state-of-the-art','invotec','iphone','ipad','smartphone','seos',
      'jobsgroupnet','languagelab.com','glycotechnology','matchik.com','webdesign','inventive','http','//www','mountvacation.co.uk',
      'mydnd.com','on-line','pharmiwebcom','internet-based','html5','themelock.com','analysts','e-learning','innovator','stuff-uk.net',
      'creatingnewclients.com','criticscom','thinkpropertycom','websphere','cloud-based','startup','optimisation',
       'optimization','interactive','visual','creation','computational','technology-based','mysql','e-trading','website/e-mail','telemarketing',
      'start-up','machine-to-machine','cryptography','pc-to-phone','easysoftware']
government1=['management','/directory/agency','industry','people','information','training','consultancy','enery','media','news','kingdom',
            'security','public','legal','healthcare','organisation','financial','markets','industries','organizations','banking','properties',
             'legal','centre','organisations','organizations','safety','strategy','infrastructure','national','telecoms','agencies',
             'intelligence','environmental','transport','airport','diseases','consultants','contract','storage','marketplace','residential',
            'telephone','artists','medicines','association','environment','planning','offshore','hospital','institute',
             'not-for-profit','radio','facilities','agents','aircraft','minerals','acquisition','chemical','rights',
            'telecom','facilities','satellite','minerals','issues','mobility','hospitals','entertainment','non-profit',
             'contracts','marine','land','tank','telecommunications','solar','cyber','sport','commission','savings','defence',
            'developments','coal','evaluation','licenses','parks','reports','carbon','recycling','commerce','aerospace','governance',
            'institutional','nursing','surgeons','strategies','employment','solicitors','diamond','leadership','mortgage',
            'fee','department','government','solicitor','emergency','consultancies','corporation','fire','bills','environments',
            'resourcing','biomass','report','contractors','report','logistics','corporates','policy','ngo','charter','countries',
             'installations','consultant','pharmaceuticals','forestry','agreements','political','lawyer','criminal','insolvency',
            'authentication','auction','rural','rehabilitation','agricultural','currency','advisors','military','anthracite',
            'anti-piracy','fossil','biomedical','committees','non-governmental','implementation','commuters','charities','communal',
            'people-powered','copper-gold','adminstrators','people-powered','attorney','arts','translators','freelancer','totism',
            'assurance','charitable','investigators','democracy','remortgage','liability','populations','politics','democratise',
            'society','timber','elderly','governed','intercultural','politicians','nonprofit','defensive']
education1=['training','research','education','learning','resources','students','knowledge','skills','courses','educational',
           'academic','oncology','essays','dissertations','tutorials','certification','teaches','ntellectual','pre-clinical',
           'educate','examination','assessment','educationalists','mathematics','researchers','academics','qualifications',
           'learners','study','e-learning','in-school','university','universities']

In [41]:
non_tech1=education1+government1+tech1
others1=[]
for x in tuple(tags1):
    if x not in non_tech1:
        others1.append(x)

In [42]:
data3['TYPE']=data3.TAGS2.apply(industry,args=(tech1,government1,education1,others1))

C:\Users\giris\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [43]:
data3.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE,TAGS2,token_words
1,@Futsal,http://futsaluk.net,Educational courses through the medium of spor...,Europe,United Kingdom,NaN,2008,Universities/Schools,"[educational, courses, through, the, medium, o...","[educational, courses, medium, sports, childre..."
3,03Numbers,http://planet-numbers.co.uk/numbers/0333,Planet Numbers are the leading provider of 03 ...,Europe,United Kingdom,NaN,2008,Unclassified,"[planet, numbers, are, the, leading, provider,...","[planet, numbers, leading, provider, 03, numbe..."
12,1000heads,https://1000heads.com/,Help brands to be more social; to talk to peop...,Europe,United Kingdom,NaN,2000,Startup,"[help, brands, to, be, more, social, ;, to, ta...","[help, brands, social, ;, talk, people, ,, tal..."
15,10Associates,http://10associates.co.uk,Over 200 Successful Re-brands,Europe,United Kingdom,NaN,2004,Unclassified,"[over, 200, successful, re-brands]","[200, successful, re-brands]"
17,111pix,http://111pix.com,Uk-based independent production and internatio...,Europe,United Kingdom,NaN,2006,Unclassified,"[uk-based, independent, production, and, inter...","[uk-based, independent, production, internatio..."


In [44]:
data3.drop(['token_words'],axis=1,inplace=True)

C:\Users\giris\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [45]:
data3.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE,TAGS2
1,@Futsal,http://futsaluk.net,Educational courses through the medium of spor...,Europe,United Kingdom,NaN,2008,Universities/Schools,"[educational, courses, through, the, medium, o..."
3,03Numbers,http://planet-numbers.co.uk/numbers/0333,Planet Numbers are the leading provider of 03 ...,Europe,United Kingdom,NaN,2008,Unclassified,"[planet, numbers, are, the, leading, provider,..."
12,1000heads,https://1000heads.com/,Help brands to be more social; to talk to peop...,Europe,United Kingdom,NaN,2000,Startup,"[help, brands, to, be, more, social, ;, to, ta..."
15,10Associates,http://10associates.co.uk,Over 200 Successful Re-brands,Europe,United Kingdom,NaN,2004,Unclassified,"[over, 200, successful, re-brands]"
17,111pix,http://111pix.com,Uk-based independent production and internatio...,Europe,United Kingdom,NaN,2006,Unclassified,"[uk-based, independent, production, and, inter..."


## Data with Tags and Taglines

In [46]:
data4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9360 entries, 0 to 11581
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   NAME         9360 non-null   object 
 1   WEBSITE      9360 non-null   object 
 2   TAGLINE      9360 non-null   object 
 3   HQ_REGION    9360 non-null   object 
 4   HQ COUNTRY   9360 non-null   object 
 5   TAGS         9360 non-null   object 
 6   LAUNCH_YEAR  9360 non-null   int64  
 7   TYPE         0 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 658.1+ KB


In [47]:
data4.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE
0,63336,http://63336.com,Ai-enabled q&a service that answers to various...,Europe,United Kingdom,mobile,2002,NaN
2,#5 Magazine,http://5mag.co,Multi-platform digital lifestyle magazines abo...,Europe,United Kingdom,publishing;branding;media;platform;entertainment,2007,NaN
4,077football News & Media,http://077football.com,"077Football - the deep-rooted, hyperlocal foot...",Europe,United Kingdom,sport;advertising;football;network;game develo...,2009,NaN
5,077football News & Media Limited,http://077football.is,"A football (soccer) centric, community-specifi...",Europe,United Kingdom,publishing;game development;social;advertising...,2009,NaN
6,1 Click Services Limited,http://1clickcloud.net,"We provide out of the box hosted solutions, ma...",Europe,United Kingdom,cloud technology,2008,NaN


In [48]:
data4['TAGS2']=data4.TAGS.str.split(';')

C:\Users\giris\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
data4.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE,TAGS2
0,63336,http://63336.com,Ai-enabled q&a service that answers to various...,Europe,United Kingdom,mobile,2002,NaN,[mobile]
2,#5 Magazine,http://5mag.co,Multi-platform digital lifestyle magazines abo...,Europe,United Kingdom,publishing;branding;media;platform;entertainment,2007,NaN,"[publishing, branding, media, platform, entert..."
4,077football News & Media,http://077football.com,"077Football - the deep-rooted, hyperlocal foot...",Europe,United Kingdom,sport;advertising;football;network;game develo...,2009,NaN,"[sport, advertising, football, network, game d..."
5,077football News & Media Limited,http://077football.is,"A football (soccer) centric, community-specifi...",Europe,United Kingdom,publishing;game development;social;advertising...,2009,NaN,"[publishing, game development, social, adverti..."
6,1 Click Services Limited,http://1clickcloud.net,"We provide out of the box hosted solutions, ma...",Europe,United Kingdom,cloud technology,2008,NaN,[cloud technology]


In [50]:
tags2=[]
for i in data4.TAGS2:
    for j in i:
        tags2.append(j)

In [51]:
c2=Counter(tags2)
print(c2.most_common())

[('software', 1821), ('media', 1072), ('mobile', 1043), ('data', 982), ('design', 974), ('subscription', 875), ('platform', 847), ('finance', 750), ('deep tech', 731), ('social', 697), ('advertising', 609), ('search engine', 589), ('cloud technology', 521), ('saas', 487), ('video', 466), ('adtech', 460), ('app', 455), ('retail', 445), ('branding', 433), ('delivery', 428), ('cleantech', 428), ('community', 419), ('e-commerce', 387), ('local', 373), ('commission', 365), ('fintech', 360), ('navigation', 357), ('regtech compliance', 354), ('consulting services', 341), ('online', 339), ('hardware', 334), ('risk management', 330), ('service', 330), ('risk', 326), ('investing', 308), ('education', 298), ('payment', 297), ('communication', 297), ('content', 297), ('compliance', 289), ('ecommerce / trading', 286), ('monitoring', 278), ('management', 271), ('banking', 270), ('biotechnology', 263), ('manufacturing', 255), ('automated technology', 250), ('testing', 244), ('social media', 243), ('a

In [52]:
tech2=['software','mobile','data','iot','immersive technologies','digital transformation','food waste','console/pc gaming','platform','nosql','deep tech','bitcoin','voip','3d printing','chatbot','search engine','cloud technology','saas','adtech','app',
      'cleantech','e-commerce','fintech','regtech compliance','online','hardware','ecommerce / trading','biotechnology',
      'automated technology','analytics','industrial technology','game','technology','big data','enterprise software',
      'game development','insurtech','information technology','digital','tech','wireless technology','machine learning',
      '3d technology','computer vision','web','online community','process automation','artificial intelligence',
      'ecommerce','gaming','website','green technology','sensor','database','recognition technology','internet',
      'data analytics','imaging technology','virtual reality','sound technology','intellectual property','computer',
      'application','technical assistance','predictive analytics','nanotech','smart tech','email','creative solutions',
      'e-learning','augmented reality','applicant tracking','market intelligence','intelligence','business intelligence',
      'legal tech','visualization','cybersecurity','innovation','fintech related','web design','iphone','cooling tech',
      'online marketing','process technologies','blockchain','android','web hosting','cloud computing','iot internetofthings',
      'network security','online learning','industrial technology','techscale200','cloud services','iot','medical technology',
      'text analytics','mobile app','speech recognition','virtual','optimization','fin-tech','agritech','animation','digital media',
      'robotic','smart city','robotic','it consulting','autonomous & sensor tech','e-mail marketing','ipad','business analytics',
      'wifi','embedding technology','influencer marketing','virtualization','search engine optimization','api','camera surveillance',
      'tablet','location analytics','edtech','algorithm','microsoft tools','digital marketing','3d printing','charging technology',
      'vivatech2019','ios','image recognition','web development','cloud storage','medical imaging','google','programming','computing',
      'deep learning','innovation management','smartphone','cryptocurrencies','mobile marketing','knowledge management',
      'web-based','startup','start-up','youtube','innovation procurements','blogging','programmatic','p2p lending',
       'cloud data services','ict services','digital services','iaas','app development','autonomus vehicles','coding',
      'restaurant tech','pc gaming','digital entertainment','video games','proptech','mobile commerce','intelligent systems',
      'smart home','cell technology','search engine optimisation','html5','transformation','online travel','ebay','behavior technology',
      'video streaming','on-demand services','browser','laser technology','interactive voice response','programmatic advertising',
      'mobile advertising','financial analytics','machine to machine','forecasting','internet of things','sales analytics',
      'encryption','paas','booking platform','bitcoin','natural language processing','jobs portal','data protection',
      'prediction','ethernet','smart grid','immersive technologies','video marketing','healthcare tech','cloud infrastructure',
      'mobile device','mobile application management','cloud-based','social data','digital transformation','it-security',
      'video chat','intranet','new product development','ecommerce infrastructure','ecommerce enablers','it support',
      'it services','cloud security','e-mail security','it management','material technology','data driven','content discovery',
      'gadget','facial recognition','social entrepreneurship','playstation','cyber security','behavioral analysis','quantum technologies',
      'high performance computing','mobile platform','mobile application','mobile games','instagram','enterprise content management',
      'e commerce','data integration','website design','metasearch','e-books','learning games','marketing automation platform',
      'web security','it support services','business software','e-cigarettes','online video advertising','indoor navigation',
      'data visualization','esignatures','online identity','online marketplace','console games','internet service providers',
      'information services','real-time feedback','learning platform','visualisation software','esignatures','online identity',
      'digital health and medical technologies','bioinformatics','online advertising','video content','online marketplace',
      'internet service providers','real-time feedback','visualisation software','binary analysis','online gaming','facebook marketing',
      'online file storage','text recognition','cooking tech','wearable technology','technical consultancy services','mobile ticketing',
      'mcommerce','virtual world','mobile technologies','entrepreneurship','iot solution','finance/fintech','it automation','home automation',
      'high-tech','hardware + software','nanotechnology','web application','digital experience management','online display advertising',
      'display marketing','digitisation','digital images','data science','healthcare analytics','digital learning','online training',
      'business transformation','social marketing','unmanned systems','online project management','it service management','app marketing',
      'virtual office','dataset','mobile media','social intelligence','social collaboration','software platform','ecommerce/trading',
      'laas','online security','internet security','mobile security','iot security','sms marketing','data services','industrial internet',
      'mobile enterprise','high tech','website development','data services','employee benefits platform','industrial internet',
      'openstack','kubernetes','mobile enterprise','high tech','salesforce','website development','digital marketing software',
      'investment platform','digital publishing','airbnb','virtual assistant','innovative food','content marketing','business network',
      'sentiment analysis','secure messaging','medical software','marketing strategy','web chat','hardware and software','network design',
      'smart lighting','industrial iot','mobile social','legal technology','online legal services','management software','email signatures',
      'neuromarketing','enterprise applications','enterprise apps','computer hardware','computer software','online scheduling',
      'low code','cash flow forecasting','fish farming','pc games','visual media','ab testing','data provider','dashboard','business dashboard',
      'smart sensor','blogging platforms','predictive targeting','semantic targeting solutions','mobile development','core algorithms',
      'programmatic advertising platform','novel farming','online services','project management software','social innovation',
      'cyber security analytics','internetofthings','communication tool','spam protection','digital health','real time data','open data',
      'online forms','call center app','customer engagement analytics','retail technology','smart device','low-code','medical simulation',
      'technology platform','e-mail management','online media marketing','media analysis','social tv','web platform','virtual room',
      'web/software agency','speech analytics','sales platform','robo advisory','wealth tech','industrial application','email newsletter',
      'mobile analytics','financial technology','skype','health care information technology','data solutions','corporate solutions',
      'legal software','google apps','smart watch','artificial','cloud-based music','advanced analytics','4g lte','digital signs',
      'medical data systems','workforce optimisation','programmatic  platform','voice api','mobile web services','website consultancy',
      'organization consulting','virtual server','virtual desktop','it solutions','python','sports betting','programmatic  platform',
      'online ads','robotic process automation','speech processing','conversational ai','automatic speech recognition','neural networks',
      'social opinion platform','infrastructure as a service','e-scooters','e-commerce platform','internet technology','energy management solutions,',
       'e-commerce platform','industrial solution','digital merchandising','ecommerce business','digital marketing campaigns','data driven marketing',
      'broadband solutions','online payment system','user interface design','ecommerce management','cloud based software','mobile testing',
      '3d technology','recognition technology','real time','embedding technology','sales analytics','encryption','renewable energy',
      'inkjet printing','ecommerce infrastructure','dsp (demand side platform)','biofuel','quantum technologies','high performance computing',
      'metasearch','learning games','console games','iot solution','human computer interaction']
govt2=['co2','nanosatellite','smoking','2g','children safety','ddos protection','aerial photography','govtech','heavy industry',
      'livestock','mobile communications','security and safety','dispute resolution','water treatment','alert system','information system',
      'public safety','association','emergency alerting','offshoring','perimeter security','gambling','military','non profit',
      'internet access','arbitration','humanitarian','water distribution','legal issue','regulatory affairs','petrochemicals',
      'pipeline inspection','hotspots','patient safety','death','wildlife','biochemical','power generation','solar parks','landline',
      'cross platform','radiography','occupational health','immersive virtual reality','heart care','diagnostic imaging','voting',
      'online auctions','strategic consulting','online media','content protection','vegetable','barristers','legal services','legal advice',
      'vulnerability management','public transportation','beach','medical healthcare','energy harvesting','fiber optics','road safety',
      'stock exchange','explosives detection','physial security','aml','social impact','public cloud','nuclear medicine','market place',
      'home security','immigration','air traffic','media monitoring','solicitors','accident','mobile networks','medical industry',
      'data center','petroleum','sea','navigation','consulting services','commission','community','risk management','service','risk',
      'communication','compliance','monitoring','banking','arts & culture','agency','entertainment','trading','reporting','mapping',
      'storage','sharing economy','telecommunications','ft 1000','water','charity','security','aviation & aerospace','health',
      'medical / healthcare','mortgages & lending','verification','market','people','asset management','jobs / recruitment','transportation',
      'sustainable development goals','administration','medical','solar energy','waste reduction','geopositioning','debt collection',
      'legal','intelligence','protection','logistic','due diligence','betting & gambling','distribution & consumption','healthcare',
      'waste management','planning','recovery','broadband','travel & tourism','patient care','governance','public sector',
      'internet services','anti money laundering','connectivity','solar','energy monitoring','carbon','drug','energy efficiency',
      'law enforcement','identity management','public sector','anti money laundering','energy saving','fraud management','weather risk management',
      'media and entertainment','personal health','safety','elderly care','currency','auction','utilities','strategy','news & magazines',
      'procurement','innovation management','marine','anti fraud','mobility','surveillance','defense','licensing','satellite',
      'pension','forensics','3g','inspection','rail','freight','energy management','disaster recovery','biological','transport',
      'communications infrastructure','public relations','budgeting','vaccines','law firm','non-profit','emergency','lawyer',
      'vintage','cannabis','carbon emissions','natural resources','healthcare services','politics','energy providers',
      'internet service providers','homeland security','information services','fraud detection','public affairs','clean energy',
      'leadership','csr','regulators','off-grid energy','electrical distribution','energy services','economy','smoking','chemical',
      'carbon footprint']
edu2=['dna sequencing','authoring','epigenetics','thesis','class','classroom','collaborative learning','nondestructive testing',
     'thermodynamics','thermal management','examination','high schools','homework','hostels','clinical development','molecular biology',
     'industrial engineering','eduacation','universities','biostimulants','bioprocessing','performance evaluation','higher  education',
     'precision medicine','biomedical discovery','primary education','student loan','cancer research','seminar','microbiology','ediscovery',
     'personal development','radiotherapy','education','biotechnology','testing','21st century skills','human resources','learning',
     'medical / healthcare','research','oncology','certification','training','student','discovery','experience','cardiology',
     'scientific','kid','school','engineering services','therapeutics','testimonials','book','drug development','educational',
     'life science','chemistry','health diagnostics','knowledge management','laboratories','skills','dermatology','talent','internship',
     'metrology','evaluation','statistics','dissertation','libraries','radiology','university','assessment','university students',
     'patenting','colleges','emerging markets','chemotherapy','neuroscience','textbooks','medicinal chemistry','learning management',
     'academy','academia','tutoring']


In [53]:
non_tech2=tech2+govt2+edu2
others2=[]
for x in tuple(tags2):
    if x not in non_tech2:
        others2.append(x)

In [54]:
data4['TYPE']=data4.TAGS2.apply(industry,args=(tech2,govt2,edu2,others2))

C:\Users\giris\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [55]:
data4.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE,TAGS2
0,63336,http://63336.com,Ai-enabled q&a service that answers to various...,Europe,United Kingdom,mobile,2002,Startup,[mobile]
2,#5 Magazine,http://5mag.co,Multi-platform digital lifestyle magazines abo...,Europe,United Kingdom,publishing;branding;media;platform;entertainment,2007,Startup,"[publishing, branding, media, platform, entert..."
4,077football News & Media,http://077football.com,"077Football - the deep-rooted, hyperlocal foot...",Europe,United Kingdom,sport;advertising;football;network;game develo...,2009,Startup,"[sport, advertising, football, network, game d..."
5,077football News & Media Limited,http://077football.is,"A football (soccer) centric, community-specifi...",Europe,United Kingdom,publishing;game development;social;advertising...,2009,Startup,"[publishing, game development, social, adverti..."
6,1 Click Services Limited,http://1clickcloud.net,"We provide out of the box hosted solutions, ma...",Europe,United Kingdom,cloud technology,2008,Startup,[cloud technology]


## Mature companies

In [56]:
data5.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE
164,ABN AMRO Commercial Finance,http://abnamrocomfin.com,"Invoice factoring, invoice discounting, and as...",Europe,United Kingdom,invoicing;accounting;finance;debt collection;m...,1988,NaN
224,Acora,https://www.acora.com,Acora | IT Managed Services & IT Consultancy,Europe,United Kingdom,it;service;consulting services,1989,NaN
268,Acuma Solutions,http://acuma.co.uk,"Acuma delivers high quality, accredited traini...",Europe,United Kingdom,technical assistance;cleantech,1985,NaN
294,Addis &,http://a-addis.co.uk,Accountancy firm based in stockport,Europe,United Kingdom,finance,1952,NaN
314,ADM Computer,http://adm-computing.co.uk,ADM Computing's bespoke IT solutions support y...,Europe,United Kingdom,software,1984,NaN


In [57]:
data6=data5[data5.TAGS.isna()]

In [58]:
data5=data5[data5.TAGS.notna()]

In [59]:
data5['TAGS2']=data5.TAGS.str.split(';')

In [60]:
data5.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE,TAGS2
164,ABN AMRO Commercial Finance,http://abnamrocomfin.com,"Invoice factoring, invoice discounting, and as...",Europe,United Kingdom,invoicing;accounting;finance;debt collection;m...,1988,NaN,"[invoicing, accounting, finance, debt collecti..."
224,Acora,https://www.acora.com,Acora | IT Managed Services & IT Consultancy,Europe,United Kingdom,it;service;consulting services,1989,NaN,"[it, service, consulting services]"
268,Acuma Solutions,http://acuma.co.uk,"Acuma delivers high quality, accredited traini...",Europe,United Kingdom,technical assistance;cleantech,1985,NaN,"[technical assistance, cleantech]"
294,Addis &,http://a-addis.co.uk,Accountancy firm based in stockport,Europe,United Kingdom,finance,1952,NaN,[finance]
314,ADM Computer,http://adm-computing.co.uk,ADM Computing's bespoke IT solutions support y...,Europe,United Kingdom,software,1984,NaN,[software]


In [61]:
def mature(x,government,education):
    if len(set(x)&set(education))>0:
        value='Universities/Schools'
    elif len(set(x)&set(government))>0:
        value='Government/Non-Profit'
    else:
        value='Mature'
    return value

In [62]:
data5['TYPE']=data5.TAGS2.apply(mature,args=(govt2,edu2))

In [63]:
data5.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE,TAGS2
164,ABN AMRO Commercial Finance,http://abnamrocomfin.com,"Invoice factoring, invoice discounting, and as...",Europe,United Kingdom,invoicing;accounting;finance;debt collection;m...,1988,Government/Non-Profit,"[invoicing, accounting, finance, debt collecti..."
224,Acora,https://www.acora.com,Acora | IT Managed Services & IT Consultancy,Europe,United Kingdom,it;service;consulting services,1989,Government/Non-Profit,"[it, service, consulting services]"
268,Acuma Solutions,http://acuma.co.uk,"Acuma delivers high quality, accredited traini...",Europe,United Kingdom,technical assistance;cleantech,1985,Mature,"[technical assistance, cleantech]"
294,Addis &,http://a-addis.co.uk,Accountancy firm based in stockport,Europe,United Kingdom,finance,1952,Mature,[finance]
314,ADM Computer,http://adm-computing.co.uk,ADM Computing's bespoke IT solutions support y...,Europe,United Kingdom,software,1984,Mature,[software]


In [64]:
data6['TAGS2']=data6.TAGLINE.apply(lambda x:word_tokenize(x))

In [65]:
data6.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE,TAGS2
594,Alver Veterinary Group,http://alvervets.co.uk,A caring and compassionate service to the animals,Europe,United Kingdom,NaN,1965,NaN,"[A, caring, and, compassionate, service, to, t..."
615,Ambivet Veterinary Group,http://ambivet.com,Dedicated small animal veterinary practice bas...,Europe,United Kingdom,NaN,1979,NaN,"[Dedicated, small, animal, veterinary, practic..."
682,Ansador,http://ansador.co.uk,Ansador is now firmly established as one of th...,Europe,United Kingdom,NaN,1982,NaN,"[Ansador, is, now, firmly, established, as, on..."
899,Aston Business School,http://aston.ac.uk,School,Europe,United Kingdom,NaN,1947,NaN,[School]
1025,Ayres Veterinary Hospital,http://ayresveterinaryhospital.co.uk,A vast range of veterinary care from vaccinati...,Europe,United Kingdom,NaN,1973,NaN,"[A, vast, range, of, veterinary, care, from, v..."


In [66]:
data6.TAGS2=data6.TAGS2.apply(lower)

In [67]:
data6['token_words']=data6.TAGS2.apply(remove_stop_words)

In [68]:
tags3=[]
for i in data6.token_words:
    for j in i:
        tags3.append(j)
print(tags3)

['caring', 'compassionate', 'service', 'animals', 'dedicated', 'small', 'animal', 'veterinary', 'practice', 'based', 'aspley', ',', 'wollaton', ',', 'heanor', ',', 'ripley', 'ilkeston', 'ansador', 'firmly', 'established', 'one', 'leading', 'fire', '&', 'security', 'companies', 'uk', 'school', 'vast', 'range', 'veterinary', 'care', 'vaccinations', 'pet', 'passports', 'small', 'animal', 'general', 'practice', ',', 'specializing', 'care', 'pet', 'animals', 'school', 'carrick', 'veterinary', 'group', 'operates', 'veterinary', 'services', 'provider', 'veterinary', 'care', 'services', 'connexion', 'world', 'cargo', ',', 'freight', 'forwarder', 'veterinary', 'care', 'services', 'veterinary', 'services', 'quality', 'veterinary', 'care', 'pets', 'halesworth', 'company', 'focuses', 'providing', 'wave', 'research', 'technologies', 'embassy', 'english', 'one', 'english', 'language', 'schools', 'world', '’', 'leading', 'provider', 'business', 'solutions', 'legal', 'structures', 'resort', 'developme

In [69]:
fdist3=FreqDist(tags3)
print(fdist3.most_common())

[(',', 14), ('veterinary', 11), ('care', 7), ('school', 6), ('services', 6), ('animal', 5), ('small', 4), ('practice', 4), ('service', 3), ('animals', 3), ('one', 3), ('leading', 3), ('pets', 3), ('providing', 3), ('dedicated', 2), ('uk', 2), ('pet', 2), ('group', 2), ('provider', 2), ('world', 2), ('english', 2), ('’', 2), ('business', 2), ('market', 2), ('agency', 2), ('people', 2), ('vet', 2), ('marketing', 2), ('equine', 2), ('farm', 2), ('soft', 2), ('drinks', 2), ('london', 2), ('referral', 2), ('caring', 1), ('compassionate', 1), ('based', 1), ('aspley', 1), ('wollaton', 1), ('heanor', 1), ('ripley', 1), ('ilkeston', 1), ('ansador', 1), ('firmly', 1), ('established', 1), ('fire', 1), ('&', 1), ('security', 1), ('companies', 1), ('vast', 1), ('range', 1), ('vaccinations', 1), ('passports', 1), ('general', 1), ('specializing', 1), ('carrick', 1), ('operates', 1), ('connexion', 1), ('cargo', 1), ('freight', 1), ('forwarder', 1), ('quality', 1), ('halesworth', 1), ('company', 1), ('

In [70]:
govt3=['care','services','service','market','agency','fire','security','vaccinations','veterinary','passports','freight',
      'embassy','legal','developments','newspaper','investigation','treatment','people']
edu3=['school','schools','cambridge','student']

In [71]:
data6['TYPE']=data6.TAGS2.apply(mature,args=(govt3,edu3))

In [72]:
data6.drop('token_words',axis=1,inplace=True)

In [73]:
data6.head()

,NAME,WEBSITE,TAGLINE,HQ_REGION,HQ COUNTRY,TAGS,LAUNCH_YEAR,TYPE,TAGS2
594,Alver Veterinary Group,http://alvervets.co.uk,A caring and compassionate service to the animals,Europe,United Kingdom,NaN,1965,Government/Non-Profit,"[a, caring, and, compassionate, service, to, t..."
615,Ambivet Veterinary Group,http://ambivet.com,Dedicated small animal veterinary practice bas...,Europe,United Kingdom,NaN,1979,Government/Non-Profit,"[dedicated, small, animal, veterinary, practic..."
682,Ansador,http://ansador.co.uk,Ansador is now firmly established as one of th...,Europe,United Kingdom,NaN,1982,Government/Non-Profit,"[ansador, is, now, firmly, established, as, on..."
899,Aston Business School,http://aston.ac.uk,School,Europe,United Kingdom,NaN,1947,Universities/Schools,[school]
1025,Ayres Veterinary Hospital,http://ayresveterinaryhospital.co.uk,A vast range of veterinary care from vaccinati...,Europe,United Kingdom,NaN,1973,Government/Non-Profit,"[a, vast, range, of, veterinary, care, from, v..."


In [74]:
frames=[data2,data3,data4,data5,data6]
result=pd.concat(frames)
result.reset_index(inplace=True)

In [75]:
result.drop('TAGS2',axis=1,inplace=True)


In [76]:
result.to_excel('Results.xlsx',sheet_name='Classification')

In [77]:
industry_count=result.TYPE.value_counts().to_frame()

In [78]:
industry_count.to_excel('Results.xlsx',sheet_name='Industry_Counts')

In [79]:
with pd.ExcelWriter('Results.xlsx') as writer:  
    result.to_excel(writer, sheet_name='Classification')
    industry_count.to_excel(writer, sheet_name='Industry_Counts')